In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
import pandas as pd

/Users/nsteins/Documents/DeepProduct/env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [38]:
labels = pd.read_csv('../Data/unbalanced_train_segments.csv',header=2, quotechar=r'"',skipinitialspace=True)

In [39]:
laugh_labels = pd.read_csv('../Data/laugh_labels.csv',names=['num','label','description'])

In [40]:
not_laugh_labels = pd.read_csv('../Data/human_non_laugh_labels.csv',names=['num','label','description'])

In [41]:
laugh_labels

,num,label,description
0,16,/m/01j3sz,Laughter
1,17,/t/dd00001,Baby laughter
2,18,/m/07r660_,Giggle
3,19,/m/07s04w4,Snicker
4,20,/m/07sq110,Belly laugh
5,21,/m/07rgt08,"Chuckle, chortle"


In [42]:
l_str = '|'.join(laugh_labels['label'].values)

In [43]:
labels['laughter'] = labels['positive_labels'].str.contains(l_str)

In [14]:
positive = labels[labels['laughter']==True]
negative = labels[labels['laughter']==False].sample(positive.shape[0])
subset = positive.append(negative)
subset.to_csv('laugh_training_subset.csv')

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subset['# YTID'],subset['laughter'],train_size=0.8)

/Users/nsteins/Documents/DeepProduct/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [16]:
files

[]

In [44]:
import glob
files = glob.glob('../Data/audioset_v1_embeddings/unbal_train/A0.tfrecord')
filename_queue = tf.train.string_input_producer(files, num_epochs=1)

In [ ]:
%%time
audio_embeddings_dict = {}
audio_labels_dict = {}

#Load embeddings
sess = tf.Session()
files = glob.glob('audioset_v1_embeddings/unbal_train/*')

examples = subset['# YTID'].values

for tfrecord in files:
    for example in tf.python_io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding = 'UTF-8')
        if vid_id in examples:
            example_label = list(np.asarray(tf_example.features.feature['labels'].int64_list.value))
            tf_seq_example = tf.train.SequenceExample.FromString(example)
            n_frames = len(tf_seq_example.feature_lists.feature_list['audio_embedding'].feature)
            
            audio_frame = []
            for i in range(n_frames):
                audio_frame.append(tf.cast(tf.decode_raw(tf_seq_example.feature_lists.feature_list['audio_embedding'].
                                                         feature[i].bytes_list.value[0],tf.uint8
                                                        ),tf.float32))
                audio_embeddings_dict[vid_id] = audio_frame
                audio_labels_dict[vid_id] = example_label

In [15]:
%%time

files = glob.glob('audioset_v1_embeddings/unbal_train/*')
subset_ids = subset['# YTID'].values

writer = tf.python_io.TFRecordWriter('bal_laugh_subset.tfrecord')
for tfrecord in files:
    for example in tf.python_io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding = 'UTF-8')
        if vid_id in subset_ids:
            writer.write(example)

writer.close()

CPU times: user 26min 31s, sys: 17.1 s, total: 26min 49s
Wall time: 27min 4s


In [50]:
%%time
labels = pd.read_csv('../Data/eval_segments.csv',header=2, quotechar=r'"',skipinitialspace=True)
labels['laughter'] = labels['positive_labels'].str.contains(l_str)
positive = labels[labels['laughter']==True]
negative = labels[labels['laughter']==False].sample(positive.shape[0])
subset = positive.append(negative)
print(subset.shape[0])
subset.to_csv('../Data/eval_laugh_training_subset2.csv')


files = glob.glob('../Data/audioset_v1_embeddings/eval/*')
subset_ids = subset['# YTID'].values

i=0
j=0
ids = []
writer = tf.python_io.TFRecordWriter('../Data/eval_laugh_subset2.tfrecord')
for tfrecord in files:
    j+=1
    for example in tf.python_io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode()
        if vid_id in subset_ids:
            writer.write(example)
            ids.append(vid_id)
            i += 1
print(f'{i} - {j}')
writer.close()

586
206 - 1444
CPU times: user 2.15 s, sys: 65.6 ms, total: 2.22 s
Wall time: 2.23 s


In [56]:
sorted(ids)[:10]

['-0BIyqJj9ZU',
 '-21_SXelVNo',
 '-53zl3bPmpM',
 '-A3zsFeU_OI',
 '-ct58kOlLfs',
 '-h5ZGw9Ci5c',
 '04xLK7R5wNA',
 '06GhXB2_XNE',
 '15Mw2jyyHk0',
 '16TsDMjHzYU']

In [57]:
sorted(subset_ids)[:10]

['-0BIyqJj9ZU',
 '-21_SXelVNo',
 '-53zl3bPmpM',
 '-A3zsFeU_OI',
 '-Igq1W20Gi8',
 '-OAyRsvFGgc',
 '-StCvzCSVdA',
 '-ct58kOlLfs',
 '-fPdOa99Iwg',
 '-fmmdkXn5YA']

In [68]:
t = list(tf.python_io.tf_record_iterator('../Data/audioset_v1_embeddings/unbal_train/-l.tfrecord'))

In [69]:
tf_ids = []
for e in t:
    tf_example = tf.train.Example.FromString(e)
    tf_ids.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode())

In [70]:
tf_ids

['-lW4JwiJaIs',
 '-l8i0MYOjrc',
 '-lctYf-HpBY',
 '-lhH_loyOhc',
 '-lMA-0qsNsc',
 '-lHnQr8IEHU',
 '-lF_IqJogos',
 '-lyHKSUqSkg',
 '-l8OBEGh6Ng',
 '-lZUwBMOU8U',
 '-lArS8I2A2U',
 '-lHF9tBL2B8',
 '-lXmddd8sZA',
 '-lrmsVd4s5c',
 '-lajVQfHJaY',
 '-ldOG91yI_k',
 '-lf86fSAHRI',
 '-l3WgUWr52k',
 '-luZdAntyq4',
 '-lTytQgzV8k',
 '-lbfLxiMvcI',
 '-loWja-5jmU',
 '-l8PxqOuQno',
 '-lCu26xGUSg',
 '-lE5yMyd2Mw',
 '-l9TqbkhlEQ',
 '-lAONbqi2HA',
 '-lg6krIj5SI',
 '-llJvWP3tSM',
 '-lmuvdsZyCw',
 '-le1KR6ybCA',
 '-lkXWMBch6s',
 '-lLnmxPL83Y',
 '-lZNaGWmsG0',
 '-lyAiroSiIs',
 '-lAkGY8ypZ4',
 '-lshbHAJMQ0',
 '-l6bYei-LxM',
 '-l16LbVPPeA',
 '-l4S333kDuw',
 '-lU2rX14m4A',
 '-lkxCUDYBnw',
 '-lrhcLGIsbU',
 '-lzlb0Yodbk',
 '-lPFROK204w',
 '-leTVEqk5nc',
 '-lw-KT0-T0Q',
 '-lxNBsqn9R0',
 '-lCM-ctNo70',
 '-l2k9T-sflA',
 '-lOlpeKgT5o',
 '-lfLIOzNuYY',
 '-lkq61mX2CY',
 '-lFU-UUwplY',
 '-lZQZ1yxJ8Q',
 '-lbKkaP8IKY',
 '-l4NAKqnRa8',
 '-l3fALTe3Nk',
 '-lBrwKRrkXc',
 '-lE2628JzrM',
 '-lhsexLHYKA',
 '-lUP2SJyUwM',
 '-lw1Ka

In [3]:
for example in tf.python_io.tf_record_iterator('bal_laugh_subset.tfrecord'):
    tf_example = tf.train.SequenceExample.FromString(example)

In [17]:
np.frombuffer(tf_example.feature_lists.feature_list['audio_embedding'].feature[0].bytes_list.value[0],np.uint8).astype(np.float32)

array([178.,  65.,  43., 163., 129., 144.,  53.,  81., 159., 138., 237.,
       143., 112.,  94., 132.,  90., 194., 143., 148.,  56., 215., 151.,
       102., 145.,  96., 126., 128.,  56., 102., 165.,  75.,  92., 162.,
       142., 129., 140., 105., 123.,  89., 119.,  95., 118., 111., 176.,
        84., 150., 197.,  62., 155., 197., 183.,   0., 171., 142.,  82.,
       142., 255.,  56., 174., 242., 180.,   0., 182., 122.,  52.,  86.,
       107., 219., 161., 235., 166.,  76.,  85.,  81.,  70., 255., 255.,
       141., 124., 255., 126., 149.,  97., 172., 102.,  83., 195.,   3.,
        33., 217.,   1., 131., 199.,  49.,  67., 153., 143.,  72., 178.,
        32., 177.,  20., 120., 150., 112., 125., 116., 106., 158., 125.,
       158., 126., 255., 118., 189.,  94., 137., 135., 193., 142., 174.,
        83., 150., 114., 255., 142., 103., 255.], dtype=float32)

## Create Balanced Laughter/speech dataset

In [32]:
n_str = '|'.join(not_laugh_labels['label'].values)
labels['not_laughter'] = (labels['positive_labels'].str.contains(n_str) & ~labels['positive_labels'].str.contains(l_str))

In [35]:
positive = labels[labels['laughter']==True]
negative = labels[labels['not_laughter']==True].sample(positive.shape[0])
subset = positive.append(negative)
subset.to_csv('../Data/laugh_speech_training_subset.csv')

In [37]:
positive.shape

(9384, 6)

In [32]:
%%time
import glob
files = glob.glob('../Data/audioset_v1_embeddings/unbal_train/*')
subset_ids = subset['# YTID'].values

writer = tf.python_io.TFRecordWriter('../Data/bal_laugh_speech_subset.tfrecord')
for tfrecord in files:
    for example in tf.python_io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding = 'UTF-8')
        if vid_id in subset_ids:
            writer.write(example)

writer.close()

CPU times: user 18min 45s, sys: 10 s, total: 18min 55s
Wall time: 19min 4s


In [34]:
%%time
labels = pd.read_csv('../Data/eval_segments.csv',header=2, quotechar=r'"',skipinitialspace=True)
labels['laughter'] = labels['positive_labels'].str.contains(l_str)
labels['not_laughter'] = (labels['positive_labels'].str.contains(n_str) & ~labels['positive_labels'].str.contains(l_str))


positive = labels[labels['laughter']==True]
negative = labels[labels['not_laughter']==True].sample(positive.shape[0])
subset = positive.append(negative)
subset.to_csv('../Data/eval_laugh_speech_training_subset.csv')
print(subset.shape[0])


files = glob.glob('../Data/audioset_v1_embeddings/eval/*')
subset_ids = subset['# YTID'].values

writer = tf.python_io.TFRecordWriter('../Data/eval_laugh_speech_subset.tfrecord')
for tfrecord in files:
    for example in tf.python_io.tf_record_iterator(tfrecord):
        tf_example = tf.train.Example.FromString(example)
        vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding = 'UTF-8')
        if vid_id in subset_ids:
            writer.write(example)

writer.close()

586
CPU times: user 3.3 s, sys: 412 ms, total: 3.71 s
Wall time: 4.67 s
